In [23]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
loader = PyPDFDirectoryLoader("us census")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
final_documents[0]

Document(metadata={'source': 'us census/acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect h

In [6]:
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

/Users/uditsevak/Desktop/Langchain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import numpy as np
np.array(huggingface_embeddings.embed_query(final_documents[0].page_content))


array([-2.49750614e-02,  7.48802116e-03,  6.07665330e-02,  2.87433639e-02,
        7.38343969e-02,  1.22540504e-01, -1.49952294e-02, -7.34638376e-03,
       -1.00141130e-01, -1.65566464e-03, -1.06490869e-02,  9.63608995e-02,
       -2.55548228e-02, -1.02058508e-01,  1.60375908e-02,  1.36243403e-02,
        1.47681180e-02, -3.22496146e-02, -2.84116473e-02,  8.82342607e-02,
       -3.01535334e-02,  2.01462768e-02, -5.24295159e-02, -3.65702272e-03,
        1.23102451e-02,  5.61479479e-03,  4.09073830e-02, -1.13198033e-03,
       -9.47217457e-03,  6.47103265e-02,  8.98134485e-02, -5.35426568e-03,
        2.55072489e-02,  1.17485831e-02,  2.94315275e-02, -4.34862487e-02,
       -3.71153690e-02,  2.39202939e-02, -9.60925967e-02,  1.52798612e-02,
       -3.21496204e-02, -4.64301407e-02, -8.10123980e-02,  1.04659691e-01,
        2.16493048e-02, -1.19758991e-03, -4.37056795e-02,  8.71711299e-02,
       -1.37320729e-02,  5.47636859e-02,  7.84215808e-04,  3.08593400e-02,
        3.65715474e-02,  

In [8]:
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [9]:
query="WHAT IS HEALTH INSURANCE COVERAGE?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [10]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x306f8b1a0> search_kwargs={'k': 3}


In [18]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)
os.environ['HUGGINGFACEHUB_API_TOKEN']=os.getenv("HUGGINGFACE_API_TOKEN")

In [19]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the health insurance coverage?"
hf.invoke(query)

'What is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insured and the insurance company. The insured pays a premium to the insurance company in exchange for the insurance company’s promise to pay for the insured’s medical expenses.\n\nWhat is the health insurance coverage?\n\nThe health insurance coverage is a contract between the insured and the insurance company. The insured pays a premium to the insurance company in exchange for the insurance company’s promise to pay for the insured'

In [21]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [24]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [25]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [26]:
query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [27]:
result = retrievalQA.invoke({"query": query})
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

8 U.S. Census Bureau
which was the state’s uninsured 
rate in 2019, the year before it 
expanded Medicaid eligibility.22
Declines in the uninsured rate in 
the majority of these states were 
related to changes in their public 
and private coverage rates. For 
seven of the states with lower 
uninsured rates in 2022, the dif-
ference was driven by increases 
in private coverage. These states 
were Florida, Kansas, Mississippi, 
North Carolina, Ohio, South 
Carolina, and Texas.
For seven states, the uninsured 
rate decrease was related to 
increases in public coverage with 
no corresponding change in the 
level of private coverage. These 
states were Alabama, California, 
Georgia, Illinois, Indiana, Michigan, 
and Oklahoma. In three states 
(Missouri, New York, and Virginia), 
it was shifts in coverage from pri-
vate to public that contributed to 
the decline in th